<a href="https://colab.research.google.com/github/andy311p/Udemy_deep_learning_pytorch/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [ ]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)
test_dataset = dsets.MNIST(root='./data', 
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# New Section

# New Section

In [ ]:
batch_size = 100
n_iters = 3000
num_epochs = int(n_iters / (len(train_dataset)/batch_size))

#iterable object
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [ ]:
class RNNModel(nn.Module):
  def __init__(self,input_dim,hidden_dim,layer_dim,output_dim):
    super(RNNModel,self).__init__()
    self.hidden_dim = hidden_dim
    self.layer_dim = layer_dim

    self.rnn = nn.RNN(input_dim,hidden_dim,layer_dim,batch_first=True, nonlinearity='relu')
    self.fc = nn.Linear(hidden_dim,output_dim)

  def forward(self,x):
    #initialize hidden state with zeros
    #[layer_dim,batch_size,hidden_dim]
    h0 = Variable(torch.zeros(self.layer_dim,x.size(0),self.hidden_dim))

    #one time step
    out,hn = self.rnn(x,h0)

    #index hidden state of last time step
    #out.size() -> 100,28,100
    #out[:,-1,:] -> 100,100
    out = self.fc(out[:,-1,:])
    #out.size() -> 100,10
    return out  

In [ ]:
#image size
input_dim = 28
hidden_dim = 100
layer_dim = 1 #number of hidden layers
output_dim=10

model = RNNModel(input_dim,hidden_dim,layer_dim,output_dim)
#computes the softmax automatically
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
seq_dim = 28

iter = 0
for epoch in range(num_epochs):
  for i, (images,labels) in enumerate(train_loader):
    images = Variable(images.view(-1,seq_dim,input_dim))
    labels = Variable(labels)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    iter += 1
    #every 500 iteration calculate accuracy
    if iter % 500 == 0:
      correct = 0.0
      total = 0.0
      #calc accuracy on test data
      for images, labels in test_loader:
        images = Variable(images.view(-1,seq_dim,input_dim) )
        outputs = model(images)
        _, predictions = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predictions == labels).sum()

      accuracy = 100 * correct / total
      print('iteration: {}. loss: {} Accuracy: {}'.format(iter, loss.data,accuracy)) 


iteration: 500. loss: 1.1484613418579102 Accuracy: 51.119998931884766
iteration: 1000. loss: 0.5741965770721436 Accuracy: 76.48999786376953
iteration: 1500. loss: 0.4019617736339569 Accuracy: 89.20999908447266
iteration: 2000. loss: 0.4355017840862274 Accuracy: 89.05999755859375
iteration: 2500. loss: 0.4763094186782837 Accuracy: 91.29000091552734
iteration: 3000. loss: 0.27872785925865173 Accuracy: 91.76000213623047


In [ ]:
print(model)

RNNModel(
  (rnn): RNN(28, 100, batch_first=True)
  (fc): Linear(in_features=100, out_features=10, bias=True)
)
